In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import TimestampType
import zipfile

from pyspark.sql.functions import monotonically_increasing_id

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1743675731502_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:

spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
new_song_data_path = "s3://data-emr-bucket-store/subset-data/song-data/datafile/*/*/*/*.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
song_data = spark.read.json(new_song_data_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
song_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+-----------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|  artist_location|artist_longitude|         artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+-----------------+----------------+--------------------+---------+---------+------------------+--------------------+----+
|ARDR4AC1187FB371A1|           NULL|                 |            NULL|Montserrat Caball...|511.16363|        1|SOBAYLL12A8C138AF9|Sono andati? Fing...|   0|
|AREBBGV1187FB523D2|           NULL|      Houston, TX|            NULL|Mike Jones (Featu...|173.66159|        1|SOOLYAZ12A6701F4A6|Laws Patrolling (...|   0|
|ARMAC4T1187FB3FA4C|       40.82624|Morris Plains, NJ|       -74.47995|The Dillinger Esc...|207.77751|        1|SOBBUGU12A8C13E95D|Setting Fire to S...|2004|
|ARPBNLO1187FB3D52F|       40.71455|     New York, N

In [6]:
songs_table = song_data["song_id", "title", "artist_id", "year", "duration", "artist_name"]
songs_table = songs_table.drop_duplicates(subset=['song_id'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
songs_table.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+--------------------+
|           song_id|               title|         artist_id|year| duration|         artist_name|
+------------------+--------------------+------------------+----+---------+--------------------+
|SOAOIBZ12AB01815BE|I Hold Your Hand ...|ARPBNLO1187FB3D52F|2000| 43.36281|            Tiny Tim|
|SOBAYLL12A8C138AF9|Sono andati? Fing...|ARDR4AC1187FB371A1|   0|511.16363|Montserrat Caball...|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|The Dillinger Esc...|
|SOBBXLX12A58A79DDA|Erica (2005 Digit...|AREDBBQ1187B98AFF5|   0|138.63138|       Eddie Calvert|
+------------------+--------------------+------------------+----+---------+--------------------+
only showing top 4 rows

In [8]:
outputfolderpath = "s3://data-emr-bucket-store/subset-data/processed-data"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
songs_table.write.partitionBy("year", "artist_id").parquet(f"{outputfolderpath}/songs_table.parquet", "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# extract columns to create artists table
artists_table = song_data["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]

# write artists table to parquet files
artists_table.write.parquet(f"{outputfolderpath}/artists_table.parquet", "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
artists_table.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+-----------------+---------------+----------------+
|         artist_id|         artist_name|  artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+-----------------+---------------+----------------+
|ARDR4AC1187FB371A1|Montserrat Caball...|                 |           NULL|            NULL|
|AREBBGV1187FB523D2|Mike Jones (Featu...|      Houston, TX|           NULL|            NULL|
|ARMAC4T1187FB3FA4C|The Dillinger Esc...|Morris Plains, NJ|       40.82624|       -74.47995|
|ARPBNLO1187FB3D52F|            Tiny Tim|     New York, NY|       40.71455|       -74.00712|
+------------------+--------------------+-----------------+---------------+----------------+
only showing top 4 rows

In [12]:
# get filepath to song data file
log_data_path = "s3://data-emr-bucket-store/subset-data/log-data/*.json"
dfLog = spark.read.json(log_data_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
dfLog.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [14]:
dfLog.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|The Big Gundown|

In [15]:
# filter by actions for song plays
df_plays = dfLog.where(dfLog.page == "NextSong")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_plays.count()
df_plays.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      

In [17]:
# extract columns for users table
# columns for users - user_id, first_name, last_name, gender, level****

users_table = df_plays["userid", "firstName", "lastName", "gender", "level", "ts"]
users_table = users_table.orderBy("ts",ascending=False).dropDuplicates(subset=["userId"]).drop('ts')

# write users table to parquet files
users_table.write.parquet(f"{outputfolderpath}/users.parquet", "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
users_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userid|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    10|   Sylvie|    Cruz|     F| free|
|   100|    Adler| Barrera|     M| free|
|   101|   Jayden|     Fox|     M| free|
|    11|Christian|  Porter|     F| free|
|    12|   Austin| Rosales|     M| free|
|    13|      Ava|Robinson|     F| free|
|    14| Theodore|  Harris|     M| free|
|    15|     Lily|    Koch|     F| paid|
|    16|    Rylan|  George|     M| paid|
|    17| Makinley|   Jones|     F| free|
|    18|    Jacob|  Rogers|     M| free|
|    19|  Zachary|  Thomas|     M| free|
|     2|  Jizelle|Benjamin|     F| free|
|    20|    Aiden| Ramirez|     M| paid|
|    22|     Sean|  Wilson|     F| free|
|    23|   Morris| Gilmore|     M| free|
|    24|    Layla| Griffin|     F| paid|
|    25|   Jayden|  Graves|     M| paid|
|    26|     Ryan|   Smith|     M| free|
|    27|   Carlos|  Carter|     M| free|
+------+---------+--------+------+-----+
only showing top

In [19]:
# create timestamp column from original timestamp column
get_datetime = udf(lambda x: datetime.fromtimestamp(int(int(x)/1000)), TimestampType())
get_weekday = udf(lambda x: x.weekday())
get_week = udf(lambda x: datetime.isocalendar(x)[1])
get_hour = udf(lambda x: x.hour)
get_day = udf(lambda x: x.day)
get_year = udf(lambda x: x.year)
get_month = udf(lambda x: x.month)

df_plays = df_plays.withColumn("start_time", get_datetime(df_plays.ts))
df_plays = df_plays.withColumn("week_day", get_weekday(df_plays.start_time))
df_plays = df_plays.withColumn("week", get_week(df_plays.start_time))
df_plays = df_plays.withColumn("hour", get_hour(df_plays.start_time))
df_plays = df_plays.withColumn("day", get_day(df_plays.start_time))
df_plays = df_plays.withColumn("year", get_year(df_plays.start_time))
df_plays = df_plays.withColumn("month", get_month(df_plays.start_time))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# extract columns to create time table
time_table = df_plays.select("start_time","hour", "day", "week", "month", "year", "week_day")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
time_table.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+---+----+-----+----+--------+
|         start_time|hour|day|week|month|year|week_day|
+-------------------+----+---+----+-----+----+--------+
|2018-11-15 00:30:26|   0| 15|  46|   11|2018|       3|
|2018-11-15 00:41:21|   0| 15|  46|   11|2018|       3|
|2018-11-15 00:45:41|   0| 15|  46|   11|2018|       3|
|2018-11-15 03:44:09|   3| 15|  46|   11|2018|       3|
|2018-11-15 05:48:55|   5| 15|  46|   11|2018|       3|
|2018-11-15 05:53:44|   5| 15|  46|   11|2018|       3|
|2018-11-15 05:55:56|   5| 15|  46|   11|2018|       3|
|2018-11-15 06:01:02|   6| 15|  46|   11|2018|       3|
|2018-11-15 06:07:37|   6| 15|  46|   11|2018|       3|
|2018-11-15 06:10:33|   6| 15|  46|   11|2018|       3|
|2018-11-15 06:13:54|   6| 15|  46|   11|2018|       3|
|2018-11-15 06:14:16|   6| 15|  46|   11|2018|       3|
|2018-11-15 06:17:59|   6| 15|  46|   11|2018|       3|
|2018-11-15 06:18:48|   6| 15|  46|   11|2018|       3|
|2018-11-15 06:21:33|   6| 15|  46|   11|2018|  

In [22]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy("year", "month").parquet(f"{outputfolderpath}/time_table.parquet", "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# songplay_id, start_time, userId, level, song_id, artist_id, sessionId, location, userAgent

df_total = song_data.join(df_plays, (song_data.title == df_plays.song) & (song_data.artist_name == df_plays.artist) )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
songplay = df_total["start_time", "userId", "level", "song_id", "artist_id", "sessionId", "location", "userAgent"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
songplay = songplay.withColumn("songplay_id", monotonically_increasing_id())
songplay.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+-----------+
|         start_time|userId|level|           song_id|         artist_id|sessionId|            location|           userAgent|songplay_id|
+-------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+-----------+
|2018-11-21 21:56:47|    15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|      818|Chicago-Napervill...|"Mozilla/5.0 (X11...|          0|
+-------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+-----------+

In [27]:
# write songplays table to parquet files partitioned by year and month
songplay.write.parquet(f"{outputfolderpath}/song_plays.parquet", "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# extract columns from joined song and log datasets to create songplays table 
log_df.createOrReplaceTempView("log_df_table")
song_df.createOrReplaceTempView("song_df_table")

songplays_table = spark.sql(
    """SELECT DISTINCT log_df_table.start_time, log_df_table.userId, log_df_table.level, log_df_table.sessionId, log_df_table.location,log_df_table.userAgent, song_df_table.song_id, song_df_table.artist_id
    FROM log_df_table 
    INNER JOIN song_df_table 
    ON song_df_table.artist_name = log_df_table.artist 
    INNER JOIN time_table
    ON time_table.start_time = log_df_table.start_time
    GROUP BY time_table.year, time_table.month
    """)
songplays_table = songplays_table.withColumn("songplay_id", monotonically_increasing_id()